In [16]:
from dotenv import load_dotenv
from langchain_mcp_adapters.client import MultiServerMCPClient
from typing import Dict, Any
from tavily import TavilyClient
from langchain.tools import tool
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langchain_community.utilities import SQLDatabase
from langchain.agents import  AgentState
from IPython.display import Markdown

load_dotenv()

True

In [3]:
client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()

In [4]:
tavily_client = TavilyClient()

In [5]:
@tool(description="Search the web for information.")
def web_search(query: str) -> Dict[str, Any]:
    return tavily_client.search(query)

In [6]:
db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")

@tool(description="Query the database for playlist information")
def query_playlist_db(query: str) -> str:
    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {str(e)}"

## **Create Subagents**

- Travel agent

In [8]:
travel_agent = create_agent(
    model="gpt-5-nano",
    tools=tools,
    system_prompt="""
        You are a travel agent. Search for flights to the desired destination wedding location.
        You are not allowed to ask any more follow up questions, you must find the best flight options based on the following criteria:
            - Price (lowest, economy class)
            - Duration (shortest)
            - Date (time of year which you believe is best for a wedding at this location)
        To make things easy, only look for one ticket, one way.
        You may need to make multiple searches to iteratively find the best options.
        You will be given no extra information, only the origin and destination. It is your job to think critically about the best options.
        Once you have found the best options, let the user know your shortlist of options.
    """
)

- Vanue Agent

In [9]:
venue_agent = create_agent(
    model="gpt-5-nano",
    tools=[web_search],
    system_prompt="""
        You are a venue specialist. Search for venues in the desired location, and with the desired capacity.
        You are not allowed to ask any more follow up questions, you must find the best venue options based on the following criteria:
            - Price (lowest)
            - Capacity (exact match)
            - Reviews (highest)
        You may need to make multiple searches to iteratively find the best options.
    """
)

- Playlist agent

In [10]:
playlist_agent = create_agent(
    model="gpt-5-nano",
    tools=[query_playlist_db],
    system_prompt="""
        You are a playlist specialist. Query the sql database and curate the perfect playlist for a wedding given a genre.
        Once you have your playlist, calculate the total duration and cost of the playlist, each song has an associated price.
        If you run into errors when querying the database, try to fix them by making changes to the query.
        Do not come back empty handed, keep trying to query the db until you find a list of songs.
        You may need to make multiple queries to iteratively find the best options.
    """
)

In [13]:
class WeddingState(AgentState):
    origin: str
    destination: str
    guest_count: str
    genre: str

## **Main Coordinator**

In [11]:
from langchain.tools import ToolRuntime
from langchain.messages import HumanMessage, ToolMessage
from langgraph.types import Command

@tool
async def search_flights(runtime: ToolRuntime) -> str:
    """Travel agent searches for flights to the desired destination wedding location."""
    origin = runtime.state["origin"]
    destination = runtime.state["destination"]
    response = await travel_agent.ainvoke({"messages": [HumanMessage(content=f"Find flights from {origin} to {destination}")]})
    return response['messages'][-1].content

@tool
def search_venues(runtime: ToolRuntime) -> str:
    """Venue agent chooses the best venue for the given location and capacity."""
    destination = runtime.state["destination"]
    capacity = runtime.state["guest_count"]
    query = f"Find wedding venues in {destination} for {capacity} guests"
    response = venue_agent.invoke({"messages": [HumanMessage(content=query)]})
    return response['messages'][-1].content

@tool
def suggest_playlist(runtime: ToolRuntime) -> str:
    """Playlist agent curates the perfect playlist for the given genre."""
    genre = runtime.state["genre"]
    query = f"Find {genre} tracks for wedding playlist"
    response = playlist_agent.invoke({"messages": [HumanMessage(content=query)]})
    return response['messages'][-1].content

@tool
def update_state(origin: str, destination: str, guest_count: str, genre: str, runtime: ToolRuntime) -> str:
    """Update the state when you know all of the values: origin, destination, guest_count, genre"""
    return Command(update={
        "origin": origin, 
        "destination": destination, 
        "guest_count": guest_count, 
        "genre": genre, 
        "messages": [ToolMessage("Successfully updated state", tool_call_id=runtime.tool_call_id)]}
        )

In [14]:
coordinator = create_agent(
    model="gpt-5-nano",
    tools=[search_flights, search_venues, suggest_playlist, update_state],
    state_schema=WeddingState,
    system_prompt="""
        You are a wedding coordinator. Delegate tasks to your specialists for flights, venues and playlists.
        First find all the information you need to update the state. Once that is done you can delegate the tasks.
        Once you have received their answers, coordinate the perfect wedding for me.
    """
)

## **Test**

In [15]:
response = await coordinator.ainvoke(
    {
        "messages": [HumanMessage(content="I'm from London and I'd like a wedding in Paris for 100 guests, jazz-genre")],
    }
)

In [17]:
Markdown(response["messages"][-1].content)

Fantastic — Paris in June with jazz vibes for 100 guests. Here’s a concise synthesis of what we have and the next steps. I’ve also started delegating to our specialists to pull together formal proposals.

What we’ve gathered so far
- Date and guest count
  - 12 June 2026, Paris, 100 guests
- Flight options (from London to Paris) — initial batch for group planning
  - Strongest value, direct options:
    - STN → CDG: 17:00–19:10, 1h10m, 53 EUR
    - SEN → CDG: 11:20–13:30, 1h10m, 58 EUR
  - Central/London-LHR options (direct, higher price):
    - LHR → CDG: ~106–113 EUR, 1h15m–1h20m
  - Other sensible options (LGW, LTN) around 77–84 EUR with short durations
- Venues (100-person capacity, Paris central areas)
  - Elegant Five-Star Parisian Hotel
    - Capacity: ~100 for a cocktail reception; wedding dinner pricing starts around 105 EUR per person (public figures)
    - Pros: Central, high-end experience; strong reviews
  - Peninsula Paris
    - Capacity: up to 100 in large ballroom for a sit-down dinner (premium tier)
    - Pros: Iconic luxury, superb service
  - InterContinental Paris Le Grand (Debussy room)
    - Capacity: 100; classic, prestigious
    - Pros: Long-standing prestige, reliable operations
  - Hotel Marignan Champs-Élysées
    - Capacity: up to 100 for cocktails; around 80 seated (layout varies)
    - Pros: Lux boutique feel, Champs-Élysées location
- Jazz playlist (initial 16-track, ~45 minutes)
  - Mix of bossa nova/romantic jazz and classic swing
  - Great for background during ceremony/cocktail; can expand for dinner and dancing

Proposed plan and next steps
1) Flights — group travel planning
- Delegate to Flight Specialist:
  - Gather group-block options for 100 guests on 12 Jun 2026
  - Prioritize direct flights from multiple London airports (LHR, LGW, STN, SEN) to CDG
  - Compare group rates, fare rules, baggage, and transfer options to the venue
  - Propose arrival windows for guests and recommended airport transfers to Paris
  - Provide 2–3 curated group-block quotes with cancellation terms and a recommended booking strategy
- Desired outcomes from you: preferred airports for the group, any airline or schedule preferences, and whether a charter/group-charter is of interest

2) Venues — shortlist, site visits, and quotes
- Delegate to Venue Specialist:
  - Confirm 100-person capacity for your preferred date and verify layout options (cocktail reception vs seated dinner)
  - Check availability for 12 Jun 2026 and potential hold dates
  - Request formal quotes and venue packages (catering minimums, bar, AV, décor, service charges, corkage, wedding coordination)
  - Assess each venue for:
    - Vibe: classic Paris luxury vs boutique chic
    - Accessibility for guests (metro/parking, crowd flow)
    - In-house catering vs outside caterer rules
    - Licensing for live music and DJ, sound restrictions, curfew
  - Prepare a 3-venue short list with pros/cons, ballpark price ranges, and next-step actions (site visit, provisional hold)
- Your input needed:
  - Which vibe resonates most (luxury classic, boutique, or a mix)?
  - Any must-haves (on-site ceremony space, private floor, outdoor terrace, skyline views, on-site spa, etc.)?

3) Jazz playlist — wedding-mood expansion
- Delegate to Playlist Specialist:
  - Extend the current jazz playlist to cover ceremony (prelude or seating), cocktail hour, dinner background, and dance floor
  - Build a 60–90 minute cocktail/dinner set plus 60–120 minute dance set, with a natural flow from romantic to upbeat
  - Include options for live jazz (quartet/trio) as an add-on and coordinate with venue acoustics and licensing needs
  - Ensure a smooth transition plan between segments and ensure licensing for streaming/DJ use
- Your input needed:
  - Do you want live jazz as an option (quintet/quartet), or strictly curated playlists?
  - Any must-have artists or songs you want included or avoided (e.g., specific eras or languages)?

What I’ll deliver next (after your confirmation)
- A refined 3-venue short list with:
  - Capacity confirmation, obstructions, and layout options
  - Availability for 12 Jun 2026, hold status, and ballpark pricing
  - Cancellation terms and payment milestones
  - Recommended next steps (site visits, tastings, and quotes)
- Flight proposals for group blocks (2–3 options) with
  - Direct vs indirect trade-offs, airports, transfer logistics
  - Group rates, baggage allowances, and booking windows
  - Arrival/departure scheduling aligned with venue timing
- A complete jazz-music plan for the day:
  - Ceremony music, cocktail hour ambiance, dinner background, and dancing set
  - Options for live vs. curated playback, with timing and licensing notes
  - A final playlist file and cue sheet for the DJ/Live musicians

How would you like to proceed?
- Confirm the date: 12 June 2026 (and any flexibility around nearby dates)
- Choose preferred London airports for the group if you have a preference (LHR, LGW, STN, SEN)
- Indicate your preferred vibe from the venues (luxury classic, boutique, or a mix)
- Tell me if you want live jazz added as part of the reception

If you’re ready, I’ll have the specialists reach out with formal proposals and hold options. I can also set up a call to review everything together and finalize the plan.